### 풀이
- 문제의 투표 규칙을 해석하면, 심사위원이 투표한 2개의 표 중 하나는 반드시 반영되어야는 내용을 `either(a, b)`로 해석할 수 있다.
- 이러한 상태에서, 1번째 참가자가 항상 참가해야 한다는 조건을 추가하는 것은 다음 절을 추가하는 것과 같다.
  - $(x_1 \lor x_1)$
  - 직관적으로는, Boolean식의 맨 뒤에 $\text{and } x_1$을 추가하는 것과 같다고 보면 될 것 같다.

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def tarjan(G):
  SCC, S, P = [], [], []
  D = [0] * len(G)
 
  s = [*range(len(G))]
  while s:
    x = s.pop()
    if x < 0:
      d = D[~x] - 1
      if P[-1] > d:
        SCC.append(S[d:])
        del S[d:], P[-1]
        for x in SCC[-1]:
          D[x] = -1
    elif D[x] > 0:
      while P[-1] > D[x]:
        P.pop()
    elif D[x] == 0:
      S.append(x)
      P.append(len(S))
      D[x] = len(S)
      s.append(~x)
      s += G[x]
  return SCC[::-1]
 
class TwoSat:
  def __init__(self, n):
    self.n = n
    self.graph = [[] for _ in range(2 * n)]
 
  def _imply(self, x, y):
    self.graph[x].append(y if y >= 0 else 2 * self.n + y)
 
  def either(self, x, y):
    """either x or y must be True"""
    self._imply(~x, y)
    self._imply(~y, x)
 
  def set(self, x):
    """x must be True"""
    self._imply(~x, x)
 
  def solve(self):
    SCC = tarjan(self.graph)
    order = [0] * (2 * self.n)
    for i, comp in enumerate(SCC):
      for x in comp:
        order[x] = i
    for i, v in enumerate(order[:self.n]):
      if v == order[~i]:
        return False, None
    return True, [+(v > order[~i]) for i, v in enumerate(order[:self.n])]

def sol(N, M) :
  ts = TwoSat(N)
  for _ in range(M):
    a, b = map(lambda x: int(x)-1 if int(x) > 0 else int(x), input().split())
    ts.either(a, b)
  ts.set(0)
  ok, ans = ts.solve()
  print("yes" if ok else "no")

while True :
  try :
    N, M = map(int, input().split())
    sol(N, M)
  except :
    break